### LangChain Expression Language

In [ ]:
from dotenv import load_dotenv
load_dotenv()

Old way

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

In [ ]:
llm = ChatOpenAI()

In [ ]:
prompt = ChatPromptTemplate.from_template("tell me about the nutritional value of {input}")

In [35]:
chain = LLMChain(llm=llm, prompt=prompt)
chain.predict(input="Pizza")

"Pizza is a popular dish that originated in Italy but is now enjoyed worldwide. Its nutritional value can vary depending on the ingredients used and the size of the portion consumed. Here is a general overview:\n\n1. Calories: The calorie content of pizza can range significantly depending on the crust, toppings, and size. On average, a slice of cheese pizza (approximately 1/8th of a 14-inch pizza) contains about 200-300 calories. However, this can increase significantly with additional toppings like meat or extra cheese.\n\n2. Carbohydrates: Pizza is typically made with a dough crust, which is a significant source of carbohydrates. Carbohydrates provide energy to the body. The specific amount can vary depending on the crust thickness and size, but a slice of cheese pizza can contain around 25-35 grams of carbohydrates.\n\n3. Protein: The protein content of pizza can vary depending on the toppings. Cheese and meat toppings like pepperoni or sausage contribute to the protein content. A s

New way

In [33]:
chain = prompt | llm
chain.invoke({"input": "Spaghetti"})

AIMessage(content="Spaghetti is a type of pasta made from wheat flour and water. It is commonly served with various sauces, vegetables, and meats. The nutritional value of spaghetti can vary depending on the serving size and the ingredients used.\n\nHere is the approximate nutritional value of 1 cup (140 grams) of cooked spaghetti without added sauce:\n\nCalories: 220\nCarbohydrates: 43 grams\nProtein: 8 grams\nFat: 1.3 grams\nFiber: 2.5 grams\nSugar: 0.9 grams\nSodium: 1 milligram\n\nSpaghetti is a good source of energy due to its high carbohydrate content. It also provides a moderate amount of protein, which is essential for muscle growth and repair. However, spaghetti is not a significant source of fat.\n\nThe fiber content in spaghetti is relatively low, but it still contributes to digestive health and can help maintain a feeling of fullness. Additionally, spaghetti is low in sugar and sodium, which is beneficial for those monitoring their intake of these nutrients.\n\nIt's importa

In [34]:
prompt.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'input': {'title': 'Input', 'type': 'string'}}}

In [36]:
chain.input_schema.schema()

{'title': 'ChainInput',
 'type': 'object',
 'properties': {'input': {'title': 'Input'}}}

### First pipe

In [37]:
from langchain.schema.output_parser import StrOutputParser

chain = prompt | llm | StrOutputParser()

In [38]:
chain = prompt | llm | StrOutputParser()
chain.invoke({"input": "Lasagna"})

"Lasagna is a traditional Italian dish that consists of layers of pasta, meat or vegetable fillings, cheese, and tomato sauce. The specific nutritional value of lasagna can vary depending on the ingredients used, portion sizes, and cooking methods. However, here is a general overview of the nutritional components of a typical serving of lasagna:\n\n1. Carbohydrates: The primary source of carbohydrates in lasagna comes from the pasta. A typical serving of lasagna (approximately 1 cup) contains around 40-45 grams of carbohydrates. The pasta provides energy in the form of complex carbohydrates.\n\n2. Protein: The protein content of lasagna largely depends on the filling used. Traditional meat lasagna contains ground beef or Italian sausage, which contributes to its protein content. Vegetarian or vegetable lasagna may include plant-based protein sources like tofu or legumes. On average, a serving of lasagna provides around 15-20 grams of protein.\n\n3. Fat: The fat content of lasagna prima

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "tell me 5 jokes about {input}"
    )

In [43]:
chain = prompt | llm.bind(stop=["\n"]) | StrOutputParser()

In [44]:
chain.invoke({"input": "pizzas"})

'Pizzas can vary widely in terms of their nutritional value depending on the ingredients used and the size of the portion consumed. Here is a general breakdown of the nutritional components typically found in a pizza:'

### OPENAI Functions

In [45]:
functions = [
    {
      "name": "joke",
      "description": "A joke",
      "parameters": {
        "type": "object",
        "properties": {
          "setup": {
            "type": "string",
            "description": "The setup for the joke"
          },
          "punchline": {
            "type": "string",
            "description": "The punchline for the joke"
          }
        },
        "required": ["setup", "punchline"]
      }
    }
  ]

In [46]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
chain = (
    prompt
    | llm.bind(function_call={"name": "joke"}, functions= functions)
    | JsonOutputFunctionsParser()
)

In [47]:
chain.invoke(input={"input": "bears"})


{'setup': "Why don't bears have fast food restaurants?",
 'punchline': "Because they can't catch the delivery truck!"}

### Working with vectorstores

In [51]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.runnable import RunnablePassthrough


vectorstore = Chroma.from_texts(["Cats are typically 9.1 kg in weight.",
                                 "Cats have retractable claws.",
                                 "A group of cats is called a clowder.",
                                 "Cats can rotate their ears 180 degrees.",
                                 "The world's oldest cat lived to be 38 years old."],
                                embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()


In [52]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [53]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [54]:
chain.invoke("how old is the oldest cat?")

'The oldest cat lived to be 38 years old.'

In [ ]:
from operator import itemgetter

In [ ]:
question = {"bla": "test", "x": "hi"}
itemgetter("bla")


In [ ]:
get_bla = itemgetter("bla")
get_bla(question)

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = {
    "context": itemgetter("question") | retriever,
    "question": itemgetter("question"),
    "language": itemgetter("language")
} | prompt | llm | StrOutputParser()

# chain = {
#     "context": (lambda x: x["question"]) | retriever,
#     "question": (lambda x: x["question"]),
#     "language": (lambda x: x["language"])
# } | prompt | llm | StrOutputParser()


In [ ]:
chain.invoke({"question": "how old is the oldest cat?", "language": "german"})

In [ ]:
!pip install duckduckgo-search

In [ ]:
from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()

In [ ]:
template = """turn the following user input into a search query for a search engine:

{input}"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
chain = prompt | llm | StrOutputParser() | search

In [ ]:
chain.invoke({"input": "whats the name of the oldest cat?"})

### Arbitrary functions

In [ ]:
from langchain.schema.runnable import RunnableLambda

def length_function(text):
    return len(text)

def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)

def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])

prompt = ChatPromptTemplate.from_template("what is {a} + {b}")

chain = {
    "a": itemgetter("foo") | RunnableLambda(length_function),
    "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")} | RunnableLambda(multiple_length_function)
} | prompt | llm | StrOutputParser()

In [ ]:
chain.invoke({"foo": "bar", "bar": "gah"})

### Interface

In [ ]:
model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
chain = prompt | model

In [ ]:
for s in chain.stream({"topic": "bears"}):
    print(s.content, end="")

In [ ]:
chain.invoke({"topic": "bears"})

In [ ]:
chain.batch([{"topic": "bears"}, {"topic": "cats"}])

In [ ]:
async for s in chain.astream({"topic": "bears"}):
    print(s.content, end="")
await chain.ainvoke({"topic": "bears"})
await chain.abatch([{"topic": "bears"}])

### Parallelism

In [ ]:
from langchain.schema.runnable import RunnableParallel
chain1 = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
chain2 = ChatPromptTemplate.from_template("tell me a bad joke about {topic}") | model
combined = RunnableParallel(joke=chain1, poem=chain2)

In [ ]:
chain1.invoke({"topic": "bears"})

In [ ]:
chain2.invoke({"topic": "bears"})

In [ ]:
combined.invoke({"topic": "bears"})

### Fallbacks

In [ ]:
from langchain.schema.output_parser import StrOutputParser

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a funny comedian and provide funny jokes about specific topics"),
        ("human", "Make a joke about {input}"),
    ]
)

fallback_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You tell the user that you currently are not able to make jokes since you are too tired"),
        ("human", "Make a joke about {input}"),
    ]
)

bad_llm = ChatOpenAI(model_name="gpt-fake")
bad_chain = chat_prompt | bad_llm | StrOutputParser()

In [ ]:
llm = ChatOpenAI()
good_chain = fallback_prompt | llm

In [ ]:
chain = bad_chain.with_fallbacks([good_chain])

In [ ]:
chain.invoke({"input": "cow"})